This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to rotation splines](index.ipynb)

# De Casteljau's Algorithm

In <cite data-cite="shoemake1985animating">Shoemake (1985)</cite>,
which famously introduces quaternions to the field of computer graphics,
Shoemake suggests to apply a variant of the
[De Casteljau's Algorithm](../euclidean/bezier-de-casteljau.ipynb)
to a quaternion control polygon,
using [Slerp](slerp.ipynb) instead of linear interpolations.

In [ ]:
def slerp(one, two, t):
    return (two * one.inverse())**t * one

In [ ]:
import numpy as np

[helper.py](helper.py)

In [ ]:
from helper import angles2quat, animate_rotations, display_animation

## "Cubic"

<cite data-cite="shoemake1985animating">Shoemake (1985)</cite>
only talks about the "cubic" case,
consisting of three nested applications of Slerp.

The resulting curve is of course not simply a polynomial of degree 3,
but something quite a bit more involved.
Therefore, we use the term "cubic" in quotes.

Shoemake doesn't talk about the "degree" of the curves at all,
they are only called "spherical Bézier curves".

In [ ]:
def de_casteljau(q0, q1, q2, q3, t):
    slerp_0_1 = slerp(q0, q1, t)
    slerp_1_2 = slerp(q1, q2, t)
    slerp_2_3 = slerp(q2, q3, t)
    return slerp(
        slerp(slerp_0_1, slerp_1_2, t),
        slerp(slerp_1_2, slerp_2_3, t),
        t,
    )

In [ ]:
q0 = angles2quat(45, 0, 0)
q1 = angles2quat(0, 0, 0)
q2 = angles2quat(-90, 90, -90)
q3 = angles2quat(-90, 0, 0)

In [ ]:
times = np.linspace(0, 1, 100)

In [ ]:
ani = animate_rotations(
    [de_casteljau(q0, q1, q2, q3, t) for t in times],
    figsize=(3, 2),
)

In [ ]:
display_animation(ani, default_mode='once')

## Arbitrary "Degree"

[splines.quaternion.DeCasteljau](../python-module/splines.quaternion.rst#splines.quaternion.DeCasteljau) class

In [ ]:
from splines.quaternion import DeCasteljau

In [ ]:
s = DeCasteljau([
    [
        angles2quat(0, 0, 0),
        angles2quat(90, 0, 0),
    ],
    [
        angles2quat(90, 0, 0),
        angles2quat(0, 0, 0),
        angles2quat(0, 90, 0),
    ],
    [
        angles2quat(0, 90, 0),
        angles2quat(0, 0, 0),
        angles2quat(-90, 0, 0),
        angles2quat(-90, 90, 0),
    ],
], grid=[0, 1, 3, 6])

In [ ]:
times = np.linspace(s.grid[0], s.grid[-1], 100)

In [ ]:
ani = animate_rotations(s.evaluate(times), figsize=(3, 2))

In [ ]:
display_animation(ani, default_mode='once')

## Constant Angular Speed

> Is there a way to construct a curve parameterized by arc length?
> This would be very useful.
>
> --<cite data-cite="shoemake1985animating">Shoemake (1985)</cite>, section 6: "Questions"

In [ ]:
from splines import ConstantSpeedAdapter

In [ ]:
s1 = DeCasteljau([[
    angles2quat(90, 0, 0),
    angles2quat(0, -45, 90),
    angles2quat(0, 0, 0),
    angles2quat(180, 0, 180),
]])

In [ ]:
s2 = ConstantSpeedAdapter(s1)

In [ ]:
ani = animate_rotations({
    'non-constant speed': s1.evaluate(np.linspace(s1.grid[0], s1.grid[-1], 100)),
    'constant speed': s2.evaluate(np.linspace(s2.grid[0], s2.grid[-1], 100)),
}, figsize=(5, 2))

In [ ]:
display_animation(ani, default_mode='reflect')

## Joining Curves

In section 4.2,
<cite data-cite="shoemake1985animating">Shoemake (1985)</cite>
provides two function definitions:

\begin{align*}
\operatorname{Double}(p, q) &= 2 (p \cdot q) q - p\\
\operatorname{Bisect}(p, q) &= \frac{p + q}{\|p + q\|}
\end{align*}

In [ ]:
def double(p, q):
    return 2 * p.dot(q) * q - p

In [ ]:
def bisect(p, q):
    return (p + q).normalize()

Given three successive key quaternions
$q_{n-1}$, $q_n$ and $q_{n+1}$,
these functions are used to compute control quaternions
$b_n$ (controlling the incoming tangent of $q_n$) and
$a_n$ (controlling the outgoing tangent of $q_n$):

\begin{align*}
a_n &=
\operatorname{Bisect}(\operatorname{Double}(q_{n-1}, q_n), q_{n+1})\\
b_n &=
\operatorname{Double}(a_n, q_n)
\end{align*}

It is unclear where these equations come from,
we only get a little hint:

> For the numerically knowledgeable, this construction
approximates the derivative at points of a sampled function by
averaging the central differences of the sample sequence.
>
> ---<cite data-cite="shoemake1985animating">Shoemake (1985)</cite>, footnote on page 249

In [ ]:
def shoemake_control_quaternions(q_1, q0, q1):
    """Shoemake's control quaternions.
    
    Given three key quaternions, return the control quaternions
    preceding and following the middle one.
    
    """
    a = bisect(double(q_1, q0), q1)
    b = double(a, q0).normalize()
    return b, a

Normalization of $b_n$ is not explicitly mentioned in the paper,
but even though the results have a length very close to `1.0`,
we still have to call `normalize()` to turn the
[Quaternion](../python-module/splines.quaternion.rst#splines.quaternion.Quaternion)
result into a
[UnitQuaternion](../python-module/splines.quaternion.rst#splines.quaternion.UnitQuaternion).

We create a helper function for building a spline,
because we will re-use the same thing further below:

In [ ]:
def create_closed_shoemake_curve(rotations):
    rotations = rotations + rotations[:2]
    control_points = []
    for q_1, q0, q1 in zip(rotations, rotations[1:], rotations[2:]):
        b, a = shoemake_control_quaternions(q_1, q0, q1)
        control_points.extend([b, q0, q0, a])
    control_points = control_points[-2:] + control_points[:-2]
    segments = list(zip(*[iter(control_points)] * 4))
    return DeCasteljau(segments)

We don't want to worry about end conditions here,
therefore we create a closed curve.

Let's come up with some example rotations
and see how Shoemake's curve-joining method works.

In [ ]:
rotations = [
    angles2quat(0, 0, 180),
    angles2quat(0, 45, 90),
    angles2quat(90, 45, 0),
    angles2quat(90, 90, -90),
    angles2quat(180, 0, -180),
    angles2quat(-90, -45, 180),
]

In [ ]:
s = create_closed_shoemake_curve(rotations)

In [ ]:
times = np.linspace(s.grid[0], s.grid[-1], 200)

In [ ]:
ani = animate_rotations(s.evaluate(times), figsize=(3, 2))

In [ ]:
display_animation(ani, default_mode='loop')

This looks a bit wobbly, doesn't it?

## Joining Curves, Second Attempt

Since Shoemake's idea doesn't seem to work that well, ...

[uniform Catmull--Rom-like quaternion splines](catmull-rom-uniform.ipynb)

[splines.quaternion.CatmullRom](../python-module/splines.quaternion.rst#splines.quaternion.CatmullRom)

In [ ]:
from splines.quaternion import CatmullRom

In [ ]:
cr = CatmullRom(rotations, endconditions='closed')

In [ ]:
ani = animate_rotations({
    "Shoemake's idea": s.evaluate(times),
    'Catmull–Rom-like': cr.evaluate(times),
}, figsize=(5, 2))

In [ ]:
display_animation(ani, default_mode='loop')

Looking at the animation of the behaviors side-by-side,
the difference should be obvious.
However, in a series of snapshots, the effect is near-impossible to see.
We can try to plot the Euler angles
in order to get a static picture that shows the different behaviors:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from helper import quat2angles

In [ ]:
def unwrap_euler(rotations):
    return np.unwrap(list(map(quat2angles, rotations)), 90, axis=0)

In [ ]:
plt.plot(times, unwrap_euler(s.evaluate(times)))
plt.title("Shoemake's idea")
plt.ylabel('angle / degrees')
plt.legend(['azimuth', 'elevation', 'roll']);

In [ ]:
plt.plot(times, unwrap_euler(cr.evaluate(times)))
plt.title('Catmull–Rom-like')
plt.ylabel('angle / degrees')
plt.legend(['azimuth', 'elevation', 'roll']);

While this doesn't really help imagining how the actual rotation looks,
it should give a clear hint towards the wobbliness of Shoemake's proposal.